In [1]:
import plotly.express as px
import pandas as pd
from time import time
from tqdm import tqdm
import numpy as np

In [2]:
import signal
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

In [12]:
from model.klotski import Klotski
game = Klotski()

a_star_results = []
with tqdm(total=3*4*4) as pbar:
    for mm in [35, 50, 65]:
        for zm in [15, 30, 45, 60]: 
            for im in [15, 30, 45, 60]:
                game.read_board('inputs/board11.txt')
                try:
                    with time_limit(25):
                        start = time()
                        result = game.a_star(mm, zm, im)
                        end = time()
                        exec_time = end - start
                        depth = len(result.move_history)
                except TimeoutException as e:
                    exec_time = np.inf
                    depth = np.inf

                a_star_results.append({
                    'mm': mm, 'zm': zm, 'im': im,
                    'exec_time': exec_time, 'result_depth': depth
                })

                pbar.update()
                
df_a_star = pd.DataFrame(a_star_results)

100%|██████████| 48/48 [17:34<00:00, 21.98s/it]


In [13]:
df = df_a_star
df[df['exec_time'] != np.inf].sort_values('exec_time', ascending=True)

,mm,zm,im,exec_time,result_depth
24,50,45,15,7.933458,96.0
44,65,60,15,8.531521,96.0
45,65,60,30,8.650076,96.0
30,50,60,45,9.532222,96.0
31,50,60,60,9.970971,96.0
29,50,60,30,11.761200,96.0
27,50,45,60,14.615383,96.0
25,50,45,30,15.044478,96.0
46,65,60,45,15.734569,96.0
6,35,30,45,19.035797,96.0
